**Дисклеймер:** весь текст этой тетрадки переведу на английский после успешной реализации.

In [1]:
import pandas as pd

In [2]:
healthcare_services = pd.read_csv('healthcare_services.csv')
renewable_energy = pd.read_csv('renewable_energy.csv')
industrial_goods = pd.read_csv('industrial_goods.csv')
fintech = pd.read_csv('fintech.csv')

## Description of the variables

## Processing of Nans/passes

In [3]:
print(f'Initial (total) amount of NaNs')
print()
print(f'In fintech data: {fintech.isna().sum().sum()}')
print(f'In industrial goods data: {industrial_goods.isna().sum().sum()}')
print(f'In renewable energy data: {renewable_energy.isna().sum().sum()}')
print(f'In healthcare services data: {healthcare_services.isna().sum().sum()}')

Initial (total) amount of NaNs

In fintech data: 208642
In industrial goods data: 209596
In renewable energy data: 94342
In healthcare services data: 178072


Сайт FMP позволяет парсить котировки американских акций только за последние 5 лет. В связи с этим создадим вспомогательные переменные

In [4]:
the_day_of_parsing = pd.to_datetime('2024-05-08', format='%Y-%m-%d')
threshold = the_day_of_parsing - pd.DateOffset(years=5)

### Fintech

Надеюсь, можно будет впоследствии запихнуть это в функцию и экстраполировать на все отрасли

In [5]:
fintech.shape

(1976, 293)

In [6]:
fintech['date'] = pd.to_datetime(fintech['date'], format='%Y-%m-%d')

In [7]:
fintech.isna().sum().sort_values(ascending=False)

close_MBNK_RU          1968
volume_MBNK_RU         1968
close_CARM_RU          1758
volume_CARM_RU         1758
volume_CRBG_USA        1562
                       ... 
Денежный агрегат М0      29
Денежный агрегат М1      29
Денежный агрегат М2      29
effr                     29
date                      0
Length: 293, dtype: int64

**Первый шаг обработки:** для удобства определения причины пропусков создадим столбец-признак `is_workday`, который принимает два значения: True в будни и False в выходные/праздники. Для этого обратимся к данным из календаря, который мы спарсили ранее.

In [8]:
calendar = pd.read_csv('data/trading_calendat_NYSE.csv')
calendar.drop(['Unnamed: 0'], axis=1, inplace=True)

In [9]:
calendar['Date'] = pd.to_datetime(calendar['Date'], format='%Y-%m-%d')
calendar['Date'].dtype

dtype('<M8[ns]')

**Первый шаг исследования:** посмотрев на таблицу, мы поняли, что надо снова отсечь данные по дате на 5 лет ранее момента парсинга, потому что FMP дает данные о котировках только за последние 5 лет.

In [10]:
#дату (месяц/день) надо менять в зависимости от момента парсинга!!

# ВТОРАЯ СТРОЧКА В ЭТОМ БЛОКЕ ПОКА НЕ РАБОТАЕТ (причина не ясна, результат плачевный)
work_calendar = calendar[calendar['Date'] > threshold]
fintech['is_workday'] = [i in work_calendar['Date'].dt.date for i in fintech['date'].dt.date]
fintech = fintech[fintech['date'] > threshold]

In [11]:
work_calendar['Date']

1346   2019-05-09
1347   2019-05-10
1348   2019-05-13
1349   2019-05-14
1350   2019-05-15
          ...    
2598   2024-04-30
2599   2024-05-01
2600   2024-05-02
2601   2024-05-03
2602   2024-05-06
Name: Date, Length: 1257, dtype: datetime64[ns]

**ПАЧИМУ((**

In [12]:
{i in work_calendar['Date'] for i in fintech['date']}

{False}

Для удобства создадим новый столбец: день недели

In [13]:
fintech['day_of_week'] = fintech['date'].dt.day_name()

In [14]:
fintech.set_index(['date'], inplace=True)
fintech.sort_index(inplace=True)

In [15]:
fintech.head(4)

,volume_NTRS_USA,close_NTRS_USA,volume_EWBC_USA,close_EWBC_USA,volume_COIN_USA,close_COIN_USA,volume_CG_USA,close_CG_USA,volume_CARM_RU,close_CARM_RU,...,Low,Close,Volume,Interbank credit market rates,gold_rates,silver_rates,platinum_rates,palladium_rates,is_workday,day_of_week
date,,,,,,,,,,,,,,,,,,,,,
2019-05-09,804008.0,96.97,748222.0,50.17,NaN,NaN,593700.0,20.76,NaN,NaN,...,NaN,NaN,NaN,6.9477,3290.00,41.27,2069.61,3303.26,False,Thursday
2019-05-10,731618.0,97.56,674296.0,50.19,NaN,NaN,405600.0,20.62,NaN,NaN,...,2511.16,2514.87,3.253956e+10,NaN,3156.00,36.81,1848.23,3472.75,False,Friday
2019-05-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,6.3560,NaN,NaN,NaN,NaN,False,Saturday
2019-05-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,6.3851,3046.02,35.11,1874.03,3836.81,False,Sunday


**Наблюдение:** после парсинга данных о `S&P500` и `imoex` от каждого появилось по 5 показателей (`high`, `low`, `open`, `close`, `volume`). Мы решили, что может быть польза именно от гэпов `high-low` и `close-open`, поэтому 5 показателей от каждого индекса можно заменить на 4: `volume`, `gap_hl = high-low`, `gap_co = close-open`, `close`.

In [16]:
fintech['S&P500_gap_hl'] = fintech['high'] - fintech['low']
fintech['S&P500_gap_co'] = fintech['close'] - fintech['open']
fintech['IMOEX_gap_hl'] = fintech['High'] - fintech['Low']
fintech['IMOEX_gap_co'] = fintech['Close'] - fintech['Open']

In [17]:
fintech.drop(['High', 'Low', 'high', 'low', 'open', 'Open'], axis=1, inplace=True)

Переименуем некоторые (схожие по названию) столбцы, чтобы не было путаницы

In [18]:
fintech.rename({'Close': 'IMOEX_close', 'close': 'S&P500_close', 'Volume': 'IMOEX_volume', 'volume': 'S&P500_volume'}, axis=1, inplace=True)

In [19]:
fintech[fintech['IMOEX_volume'].isnull()].sample(5)

,volume_NTRS_USA,close_NTRS_USA,volume_EWBC_USA,close_EWBC_USA,volume_COIN_USA,close_COIN_USA,volume_CG_USA,close_CG_USA,volume_CARM_RU,close_CARM_RU,...,gold_rates,silver_rates,platinum_rates,palladium_rates,is_workday,day_of_week,S&P500_gap_hl,S&P500_gap_co,IMOEX_gap_hl,IMOEX_gap_co
date,,,,,,,,,,,,,,,,,,,,,
2023-06-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5250.65,62.95,2632.92,3679.10,False,Saturday,NaN,NaN,NaN,NaN
2023-01-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4307.90,52.86,2300.58,3783.02,False,Saturday,NaN,NaN,NaN,NaN
2023-05-09,925407.0,72.14,1437519.0,44.88,9641197.0,60.21,3366100.0,26.03,NaN,NaN,...,6028.12,76.56,3053.59,3811.55,False,Tuesday,1.4006,-0.2,NaN,NaN
2020-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,False,Sunday,NaN,NaN,NaN,NaN
2022-08-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,False,Sunday,NaN,NaN,NaN,NaN


**Наблюдение:** большинство данных не спарсилось именно за выходные. это объясняется тем, что (почти) все отдыхают. Про праздники не будет сразу делать таких выводов, потому что там все зависит от организации.

**Решение:** избавимся от этих данных, с ними кашу не сваришь.

In [20]:
fintech = fintech[(fintech['day_of_week'] != 'Sunday') & (fintech['day_of_week'] != 'Saturday')]

In [21]:
fintech.isna().sum().sum()

39657

In [22]:
fintech.shape

(1321, 292)

In [41]:
fintech.isna().sum().sort_values(ascending=False)

gold_rates                371
palladium_rates           371
usdrub                    371
silver_rates              371
platinum_rates            371
                         ... 
Денежный агрегат М1         6
Денежный агрегат М2         6
Широкая денежная масса      6
is_workday                  0
day_of_week                 0
Length: 266, dtype: int64

Также надо не забыть, что если парсинг американских акций был не сегодня, то надо отсечь данные по значению переменной `the_day_of_parsing`.

In [24]:
fintech = fintech[fintech.index <= the_day_of_parsing]

In [25]:
fintech.isna().sum().sum()

35105

In [26]:
fintech.shape

(1305, 292)

In [27]:
fintech.isna().sum().sort_values(ascending=False)

volume_MBNK_RU         1298
close_MBNK_RU          1298
volume_CARM_RU         1088
close_CARM_RU          1088
volume_CRBG_USA         891
                       ... 
Денежный агрегат М1       6
Денежный агрегат М2       6
effr                      6
is_workday                0
day_of_week               0
Length: 292, dtype: int64

**Замечание:** заметим, что по акциям некоторых компаний практически нет данных. Так случилось из-за того, что конкретно те сайты, откуда мы доставали котировки, не хранили их по этим компаниям, может, компании слишком непопулярны и тд 

**400** взято из головы для примера - ЗАМЕНИТЬ!

In [28]:
exceptions = list(fintech.isna().sum()[fintech.isna().sum()>400].index)

In [29]:
fintech = fintech.loc[:, ~fintech.columns.isin(exceptions)]

In [37]:
fintech.isna().sum().sort_values(ascending=False)

gold_rates                371
palladium_rates           371
usdrub                    371
silver_rates              371
platinum_rates            371
                         ... 
Денежный агрегат М1         6
Денежный агрегат М2         6
Широкая денежная масса      6
is_workday                  0
day_of_week                 0
Length: 266, dtype: int64

In [31]:
fintech.isna().sum().sum()

16330

**Замечание:** если исследовать возможные причины пропусков в `usdrub`, `gold_rates`, `silver_rates`, `platinum_rates` и `palladium_rates`, то стоит обратить внимание, что в 205 из 311 понедельников этих курсов не было. Не уверен, но кажется, что прикол в том, что ЦБ в пн пользуется курсов за пт **(подтверждение не смог почему-то найти)**. Но на этот день недели есть явное смещение, думаю, для пн можно воспользоваться `fill forward`.

In [32]:
fintech[~(fintech['usdrub'].isnull()) & (fintech['day_of_week'] == 'Monday')].shape

(56, 266)

In [33]:
fintech[(fintech['usdrub'].isnull()) & (fintech['day_of_week'] == 'Monday')].shape

(205, 266)

In [34]:
fintech['usdrub'].isna().sum()

371

In [35]:
mask = (fintech['usdrub'].isnull()) & (fintech['day_of_week'] == 'Monday')
fintech.loc[mask, 'usdrub'].ffill(inplace=True)
fintech.loc[mask, 'gold_rates'] = fintech.loc[mask, 'gold_rates'].ffill()
fintech.loc[mask, 'silver_rates'] = fintech.loc[mask, 'silver_rates'].ffill()
fintech.loc[mask, 'platinum_rates'] = fintech.loc[mask, 'platinum_rates'].ffill()
fintech.loc[mask, 'palladium_rates'] = fintech.loc[mask, 'palladium_rates'].ffill()

In [36]:
fintech['usdrub'].isna().sum()

371